<a href="https://colab.research.google.com/github/SamLiu666/AI/blob/master/tf2tuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fully-connected Network

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

In [ ]:
def mnist_dataset():
  (x, y), _ = datasets.mnist.load_data() # Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

def prepare_mnist_features_and_labels(x,y):
  x = tf.cast(x, tf.float32)/255.0
  y = tf.cast(y, tf.int64)
  return x, y

In [ ]:
model = keras.Sequential([
    layers.Reshape(target_shape=(28*28,), input_shape=(28,28)),
    layers.Dense(units=100,activation='relu'),
    layers.Dense(units=100,activation='relu'),
    layers.Dense(10)])

optimizer = optimizers.Adam()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
@tf.function
def compute_loss(logits, labels):
  return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits=logits, labels=labels
  ))

@tf.function
def compute_accuracy(logits, labels):
  predictions = tf.argmax(logits, axis=1)
  return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):

  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(logits, y)

  # compute gradient
  grads = tape.gradient(loss, model.trainable_variables)
  # update to weights
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  accuracy = compute_accuracy(logits, y)

  # loss and accuracy is scalar tensor
  return loss, accuracy

def train(epoch, model, optimizer):

  train_ds = mnist_dataset()
  loss = 0.0
  accuracy = 0.0
  for step, (x, y) in enumerate(train_ds):
    loss, accuracy = train_one_step(model, optimizer, x, y)

    if step % 500 == 0:
      print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

  return loss, accuracy

In [ ]:
for epoch in range(20):
  loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3244712 ; accuracy 0.09
epoch 1 : loss 0.28798357 ; accuracy 0.91
epoch 2 : loss 0.19936447 ; accuracy 0.95
epoch 3 : loss 0.08297668 ; accuracy 0.98
epoch 4 : loss 0.06563656 ; accuracy 0.98
epoch 5 : loss 0.08349426 ; accuracy 0.97
epoch 6 : loss 0.013548888 ; accuracy 1.0
epoch 7 : loss 0.042788964 ; accuracy 0.99
epoch 8 : loss 0.055909798 ; accuracy 0.99
epoch 9 : loss 0.03208171 ; accuracy 0.99
epoch 10 : loss 0.016497966 ; accuracy 1.0
epoch 11 : loss 0.018052388 ; accuracy 1.0
epoch 12 : loss 0.0054515842 ; accuracy 1.0
epoch 13 : loss 0.0056006964 ; accuracy 1.0
epoch 14 : loss 0.0038491108 ; accuracy 1.0
epoch 15 : loss 0.0049080546 ; accuracy 1.0
epoch 16 : loss 0.0028577477 ; accuracy 1.0
epoch 17 : loss 0.0027239176 ; accuracy 1.0
epoch 18 : loss 0.0072756577 ; accuracy 1.0
epoch 19 : loss 0.0006254496 ; accuracy 1.0
Final epoch 19 : loss 0.0065804925 ; accuracy 1.0
